In [33]:
from random import randint

def totales(x, c, v):
    total_costo = sum([x[i] * c[i] for i in range(len(x))])
    total_valor = sum([x[i] * v[i] for i in range(len(x))])
    return total_costo, total_valor

def costo_miope(c, v):
    vc = [(v[i] / c[i], i) for i in range(len(c))]  # Calcula las relaciones valor/costo
    vc.sort(reverse=True)  # Ordena las relaciones de mayor a menor
    return vc

def generar_vecinos(B, v, c, x, metodo):
    m = x.copy()  # Copia la solución inicial

    if metodo == 'valor':
        ordenados = costo_miope(c, v)
    elif metodo == 'cardinalidad':
        ordenados = [(x[i], i) for i in range(len(m))]
        ordenados.sort()
    else:
        raise ValueError("Método de construcción no válido")

    for _, i in ordenados:
        # Calcula cuántas veces se puede agregar el artículo sin superar la capacidad
        veces_a_agregar = min(B // c[i], 2)

        # Verifica si hay suficiente capacidad para agregar el artículo
        if veces_a_agregar > 0:
            # Agrega el artículo la cantidad de veces calculada
            m[i] += veces_a_agregar

            # Actualiza la capacidad de la mochila
            B -= veces_a_agregar * c[i]

    return m


def busqueda_local(B, v, c, solucion_inicial):
    mejor_solucion = solucion_inicial.copy()
    mejor_valor = sum([mejor_solucion[i] * v[i] for i in range(len(mejor_solucion))])

    for i in range(len(solucion_inicial)):
        if solucion_inicial[i] > 0:
            nueva_solucion = solucion_inicial.copy()
            nueva_solucion[i] -= 1

            # Calcula cuántas veces se puede agregar el artículo sin superar la capacidad
            veces_a_agregar = min(B // c[i], 2)

            # Verifica si hay suficiente capacidad para agregar el artículo
            if veces_a_agregar > 0:
                nueva_solucion[i] += veces_a_agregar
                B -= veces_a_agregar * c[i]

                nuevo_valor = sum([nueva_solucion[i] * v[i] for i in range(len(nueva_solucion))])
                if nuevo_valor > mejor_valor:
                    mejor_solucion = nueva_solucion.copy()
                    mejor_valor = nuevo_valor

    return mejor_solucion



def grasp(B, v, c, max_iter, metodo_construccion):
    mejor_solucion = None
    mejor_valor = 0
    for _ in range(max_iter):
        solucion = generar_vecinos(B, v, c, [0] * len(c), metodo_construccion)
        costo, valor = totales(solucion, c, v)
        if costo <= B and valor > mejor_valor:
            # Aplica búsqueda local a la solución actual
            solucion_local = busqueda_local(B, v, c, solucion)
            costo_local, valor_local = totales(solucion_local, c, v)
            if costo_local <= B and valor_local > mejor_valor:
                mejor_solucion = solucion_local.copy()
                mejor_valor = valor_local
        elif mejor_solucion is None:
            mejor_solucion = solucion.copy()
            mejor_valor = valor
    return mejor_solucion

# Ejemplo de uso
c = [2, 3, 3, 5, 7]  # Costos de los elementos
v = [5, 9, 8, 3, 2]  # Valores de los elementos
B = 105  # Capacidad de la mochila
max_iter = 1000  # Número máximo de iteraciones para GRASP

# Método de construcción por valor antes de la búsqueda local
metodo_construccion_valor = 'valor'
solucion_valor = grasp(B, v, c, max_iter, metodo_construccion_valor)
if solucion_valor is not None:
    print("Solución por valor:", solucion_valor)
    print("Costo:", sum([solucion_valor[i] * c[i] for i in range(len(solucion_valor))]))
    print("Valor:", sum([solucion_valor[i] * v[i] for i in range(len(solucion_valor))]))
else:
    print("No se encontró una solución factible por valor.")

# Método de construcción por cardinalidad antes de la búsqueda local
metodo_construccion_cardinalidad = 'cardinalidad'
solucion_cardinalidad = grasp(B, v, c, max_iter, metodo_construccion_cardinalidad)
if solucion_cardinalidad is not None:
    print("Solución por cardinalidad:", solucion_cardinalidad)
    print("Costo:", sum([solucion_cardinalidad[i] * c[i] for i in range(len(solucion_cardinalidad))]))
    print("Valor:", sum([solucion_cardinalidad[i] * v[i] for i in range(len(solucion_cardinalidad))]))
else:
    print("No se encontró una solución factible por cardinalidad.")

Solución por valor: [2, 3, 2, 2, 2]
Costo: 43
Valor: 63
Solución por cardinalidad: [2, 3, 2, 2, 2]
Costo: 43
Valor: 63
